# nbdev is getting a little %magic
> Introducing magic flags in nbdev.

- toc: true 
- badges: true
- comments: true
- categories: [nbdev, fastai, jupyter]
- author: Peter Butterfill

# Overview

nbdev is progressively getting magic commands that will eventually replace the special comments it currently uses, so you get autocomplete and documentation:

<img alt="magic animation" width="75%" align="left" src="images/nbdev-magic.gif" />

You'll also get feedback when flags are used incorrectly:

<img alt="magic usage error"  width="75%" align="left" src="images/nbdev-magic-usage-error.png" />

# How do I start using magic flags?

Grab an [editable install of nbdev](https://github.com/fastai/nbdev/#installing) then run `nbdev_upgrade` from the command line - this will update notebooks that use comment flags like:

```python
#export special.module
```

to use magic flags:

```python
%nbdev_export special.module
```

To make magic flags work, `nbdev_upgrade` might need to add a new code cell to the top of the notebook:

```python
from nbdev import *
```

You can run `nbdev_upgrade` any number of times - which means you can update the same project every time new magic flags get added to nbdev.

## Can I use both comment and magic flags?

Both comment and magic flags are currently supported and you can use both kinds in the same notebook.

## Why can't I see any "test" flags?

If you don't see the test flags you need ...

<img alt="auto complete no test" width="55%" align="left" src="images/tab-complete-no-test.png" />

... you'll probably need to update `settings.ini` then restart your notebook.

Test flags are configured in your `settings.ini` (set `tst_flags`, separating flags by a `|` if you have several of them).

When the `nbdev.flags` module is imported, test flags are created dynamically from your settings.ini. If `tst_flags=slow|gpu`, the following flags would be available:
```python
%nbdev_slow_test
%nbdev_gpu_test
```

## Can I use flags in any kind of cell?

Apart from `#hide` and `%nbdev_hide`, nbdev will ignore flags that are not in code cells.
This means that you can use markdown cells for "developer-only" details and `#hide` them from the HTML docs.

## Do magic flags work differently to comment flags?

Ideally, they would work the same but ... there is a difference when nbdev splits the source of a cell into flags and code;

If no magic flags are found, treat the first comment line as a flag

<img alt="split_flags_and_code example" width="450" align="left" src="images/split_flags_and_code.png" />

If magic flags are found, the flags part can contain multiple lines

<img alt="split_flags_and_code example" width="450" align="left" src="images/split_flags_and_code_magic.png" />

This could make a difference because nbdev writes just the code part to both your library and HTML docs.

# How do comment flags correspond to magic flags?

| Comment flag                           | Magic flag                  |                                                                                |
|----------------------------------------|-----------------------------|--------------------------------------------------------------------------------|
| `default_exp`                          | `nbdev_default_export`      | Define the name of the module everything should be exported in                 |
| `exports`                              | `nbdev_export_and_show`     | Export and show code in the docs                                               |
| `exporti`                              | `nbdev_export_internal`     | Export but don’t show in docs and don’t add to `__all__`                       |
| `export`                               | `nbdev_export`              | Export but don’t show in docs                                                  |
| `hide_input`                           | `nbdev_hide_input`          | Do not show input of a test cell in docs                                       |
| `hide_output`                          | `nbdev_hide_output`         | Do not show output of a test cell in docs                                      |
| `hide`                                 | `nbdev_hide`                | Do not show a test cell or markdown in docs                                    |
| `default_cls_lvl`                      | `nbdev_default_class_level` | Define the default toc level of classes                                        |
| `collapse_output` or `collapse-output` | `nbdev_collapse_output`     | Inlcude output in the docs under a collapsable element                         |
| `collapse_show` or `collapse-show`     | `nbdev_collapse_input open` | Inlcude intput in the docs under a collapsable element that is open by default |
| `collapse_hide` or `collapse-hide`     | `nbdev_collapse_input`      | Inlcude intput in the docs under a collapsable element                         |
| `collapse`                             | `nbdev_collapse_input`      | Inlcude intput in the docs under a collapsable element                         |

# Why might we add magic flags for `_all_` and `show_doc`?

One of the reasons for suggesting this is so that everything can be done via magic flags - rather than using flags for everything except `_all_` and `show_doc`.

#hide
## demo magic flag impl

In [1]:
#hide
from IPython.core.magic import register_line_magic
@register_line_magic
def nbev_add2__all__(line):
    """To add something to __all__ if it's not picked automatically,
    write a cell with something like: %nbev_add2__all__ thing_1, thing_2 ..."""

In [2]:
#hide
from nbdev.showdoc import show_doc
@register_line_magic
def nbev_show_doc(line):
    """Show documentation for element. Supported types: class, function, and enum.
    You can use this magic flag or the `show_doc` function from `nbdev.showdoc`"""
#     show_doc(globals()[line.strip()])
    # we could let the magic show docs for class + any number of methods
    elts = line.split()
    first_elt = eval(elts[0])
    show_doc(first_elt)
    for i in range(1,len(elts)):
        show_doc(eval(elts[0]+'.'+elts[i]))
    # or parse line for other show_doc arguments?
#     show_doc(eval(line)) 

## %nbev_add2__all__

here's what we do today

In [3]:
_all_ = ['progress_bar','master_bar']

but we could ...

In [4]:
%nbev_add2__all__ progress_bar,master_bar

This flag could
- check that everything exists and names are valid (when the cell is executed) and 
- make it possible to tab-complete the items you're adding to `__all__`

<img alt="add to all"  width="55%" align="left" src="images/add2all.png" />

The example above uses a line magic, so you'd have to move everything on line 3 up to line 2 but ... we could use a cell magic

## %nbev_show_doc

here's what we do today

In [5]:
show_doc(ArithmeticError)

<h2 id="ArithmeticError" class="doc_header"><code>class</code> <code>ArithmeticError</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>ArithmeticError</code>() :: `Exception`

Base class for arithmetic errors.

In [6]:
show_doc(ArithmeticError.mro)

<h4 id="ArithmeticError.mro" class="doc_header"><code>ArithmeticError.mro</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>ArithmeticError.mro</code>()

Return a type's method resolution order.

In [7]:
show_doc(ArithmeticError.__init__)

<h4 id="ArithmeticError.__init__" class="doc_header"><code>ArithmeticError.__init__</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>ArithmeticError.__init__</code>(**\*`args`**, **\*\*`kwargs`**)

Initialize self.  See help(type(self)) for accurate signature.

but we could ...

In [8]:
%nbev_show_doc ArithmeticError mro __init__

<h2 id="ArithmeticError" class="doc_header"><code>class</code> <code>ArithmeticError</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>ArithmeticError</code>() :: `Exception`

Base class for arithmetic errors.

<h4 id="ArithmeticError.mro" class="doc_header"><code>ArithmeticError.mro</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>ArithmeticError.mro</code>()

Return a type's method resolution order.

<h4 id="ArithmeticError.__init__" class="doc_header"><code>ArithmeticError.__init__</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>ArithmeticError.__init__</code>(**\*`args`**, **\*\*`kwargs`**)

Initialize self.  See help(type(self)) for accurate signature.

This flag could make it possible to "show doc" a class and any number of its methods with a single line of code.